<a href="https://colab.research.google.com/github/sccmst/NLUModelOnColab/blob/GPT2-ContentExtension/GPT-ContentExtension/Fine_tune_GPT_2_Model_with_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
with open("/root/.kaggle/kaggle.json", "w") as f:
  f.write('{"username":"cstsmc","key":"41599cdf1be98f2d65480249a887043b"}')

!chmod 600 /root/.kaggle/kaggle.json

## **What are we going to do:**

- load the dataset from kaggle
- prepare the dataset and build a ``TextDataset``
- load the pre-trained GPT-2 model and tokenizer
- initialize ``Trainer`` with ``TrainingArguments``
- train and save the model
- test the model

In [ ]:
!pip install transformers>=4.2.2

In [ ]:
from transformers import set_seed
# Set seed for reproducibility.
set_seed(123)


In [ ]:
!nvidia-smi

Mon Dec 19 01:35:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare the dataset and build a ``TextDataset``

The next step is to extract the instructions from all recipes and build a `TextDataset`. The `TextDataset` is a custom implementation of the [Pytroch `Dataset` class](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class) implemented by the transformers library. If you want to know more about Dataset in Pytroch you can check out this [youtube video](https://www.youtube.com/watch?v=PXOzkkB5eH0&ab_channel=PythonEngineer).

First, we are going to split the `recipes.json` into a `train` and `test` section and extract `Instructions` from the recipes and write them into a `train_dataset.txt` and `test_dataset.txt`

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
#api.dataset_download_files("sterby/german-recipes-dataset",path="./", unzip=True)
api.dataset_download_files("terrychanorg/chinese-simplified-xlsum-v2", path="./", unzip=True)

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


with open('./chinese_traditional_XLSum_v2.0/chinese_traditional_train.jsonl') as f:
    # row = json.load(f)
    data = []
    for line in f.readlines():
      line.replace("\n","")
      data.append(json.loads(line))

def build_text_files(data_json, dest_path):
    with open(dest_path, 'w') as f:
      data = []
      for texts in data_json:
          title = str(texts['title']).strip()
          text = str(texts['text']).strip()
          summary = str(texts['summary']).strip()
          data.append(f"{summary}BEG;END{text}")
          data.append(f"{title}BEG;END{text}")
      f.write("\n".join(data))



In [ ]:
train, test = train_test_split(data,test_size=0.02)
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')
print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 36625
Test dataset length: 748


the next step is to download the tokenizer, which we use. We use the tokenizer from the `german-gpt2` model on [huggingface](https://huggingface.co/anonymous-german-nlp/german-gpt2).

In [ ]:
MODEL_NAME = "theta/gpt2-reporter"
# MODEL_NAME = "uer/gpt2-chinese-cluecorpussmall"
# MODEL_NAME = "ckiplab/gpt2-base-chinese"
# MODEL_NAME = "gpt2"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset
from typing import Any, Optional
import torch
import os
import logging
from filelock import FileLock
import time
import pickle
logger = logging.getLogger()
class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: Any,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):

        if os.path.isfile(file_path) is False:
            raise ValueError(f"Input file path {file_path} not found")

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            f"cached_lm_{tokenizer.__class__.__name__}_{block_size}_{filename}",
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    # text = f.read()
                    texts = f.readlines()

                # tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
                # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    # self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size])
                # )

                for text in texts:
                  tokenedtext = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
                  for i in range(0, len(tokenedtext) - block_size + 1, block_size):
                    self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenedtext[i : i + block_size]))
                
                
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    f"Saving features into cached file {cached_features_file} [took {time.time() - start:.3f} s]"
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)
        # return self.examples[i]


In [ ]:
from transformers import DataCollatorForLanguageModeling
# from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
# https://github.com/huggingface/transformers/blob/main/src/transformers/data/datasets/language_modeling.py
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=512)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=512)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (1674 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
train_dataset[:3]

tensor([[ 101, 4919, 3696,  ..., 2692, 1456,  102],
        [ 101, 5865, 1762,  ...,  511,  679,  102],
        [ 101, 6882, 8024,  ..., 3556, 1759,  102]])

# login to huggingface

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained(MODEL_NAME)


training_args = TrainingArguments(
    output_dir="./gpt2-reporter", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    evaluation_strategy="steps",
    eval_steps = 400, # Number of update steps between two evaluations.
    logging_steps= 400,
    save_steps = 800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    load_best_model_at_end=True,
    save_total_limit=3,
    learning_rate=5e-5,
    push_to_hub=True,
    hub_model_id="theta/gpt2-reporter"
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

Cloning https://huggingface.co/theta/gpt2-reporter into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.43k/401M [00:00<?, ?B/s]

...

Download file runs/Dec13_12-34-11_b897c6e75aee/events.out.tfevents.1670934865.b897c6e75aee.77.0:  78%|#######7…

Clean file runs/Dec13_12-34-11_b897c6e75aee/events.out.tfevents.1670934865.b897c6e75aee.77.0:  25%|##4       |…

Download file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

Clean file training_args.bin:  30%|##9       | 1.00k/3.37k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/401M [00:00<?, ?B/s]

# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 127374
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 31844
  Number of trainable parameters = 102068736


Step,Training Loss,Validation Loss
400,2.532500,2.536140
800,2.397500,2.536006


***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-800
Configuration saved in ./gpt2-reporter/checkpoint-800/config.json
Model weights saved in ./gpt2-reporter/checkpoint-800/pytorch_model.bin


Step,Training Loss,Validation Loss
400,2.532500,2.536140
800,2.397500,2.536006
1200,2.625700,2.519763
1600,2.611400,2.510158
2000,2.603600,2.502393
2400,2.591300,2.498073
2800,2.582600,2.488771
3200,2.578200,2.485377
3600,2.562700,2.480705
4000,2.552400,2.476747


***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-1600
Configuration saved in ./gpt2-reporter/checkpoint-1600/config.json
Model weights saved in ./gpt2-reporter/checkpoint-1600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-2400
Configuration saved in ./gpt2-reporter/checkpoint-2400/config.json
Model weights saved in ./gpt2-reporter/checkpoint-2400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2484
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-3200
Configuration saved in ./gpt2-reporter/checkpoint-3200/config.json
Model weights saved in ./gpt2-reporter/checkpoint-3200/p

KeyboardInterrupt: ignored

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()

Saving model checkpoint to ./gpt2-reporter
Configuration saved in ./gpt2-reporter/config.json
Model weights saved in ./gpt2-reporter/pytorch_model.bin
Saving model checkpoint to ./gpt2-reporter
Configuration saved in ./gpt2-reporter/config.json
Model weights saved in ./gpt2-reporter/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/401M [00:00<?, ?B/s]

Upload file runs/Dec19_01-40-54_e9fb958e7894/events.out.tfevents.1671414225.e9fb958e7894.74.0:  40%|####      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/theta/gpt2-reporter
   b7f15ec..ebe0d6a  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/theta/gpt2-reporter
   b7f15ec..ebe0d6a  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
To https://huggingface.co/theta/gpt2-reporter
   ebe0d6a..3fea100  main -> main

   ebe0d6a..3fea100  main -> main



In [ ]:
# trainer.push_to_hub()

Saving model checkpoint to ./gpt2-reporter
Configuration saved in ./gpt2-reporter/config.json
Model weights saved in ./gpt2-reporter/pytorch_model.bin
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


# Test the model

To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [ ]:
from transformers import pipeline, AutoModelWithLMHead
MODEL_NAME = 'uer/gpt2-chinese-cluecorpussmall'
model = AutoModelWithLMHead.from_pretrained('theta/gpt2-reporter')
reporter = pipeline('text-generation',model=model, tokenizer=MODEL_NAME)



/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--theta--gpt2-reporter/snapshots/3fea100c6ad27950e4591c4b173463ea4387a28a/config.json
Model config GPT2Config {
  "_name_or_path": "theta/gpt2-reporter",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--theta--gpt2-reporter/snapshots/3fea100c6ad27950e4591c4b173463ea4387a28a/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at theta/gpt2-reporter.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/config.json
Model config GPT2Config {
  "_name_or_path": "uer/gpt2-chinese-cluecorpussmall",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "in

In [ ]:
reporter('總統宣布國防預算大漲BEG;END',max_length=800)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '總統宣布國防預算大漲BEG;END 在 接 受 《 紐 約 時 報 》 採 訪 時 他 的 言 論 還 沒 有 得 到 國 防 預 算 的 支 持 ， 中 國 軍 隊 也 沒 有 取 得 什 麼 實 質 上 的 行 動 。 這 個 在 美 國 就 職 的 特 朗 普 前 任 和 聯 邦 警 察 高 級 別 官 員 早 在 2009 年 就 曾 公 開 警 告 說 ， 中 國 可 能 會 借 美 國 的 國 家 安 全 利 益 以 「 威 脅 和 操 弄 」 。 據 一 些 媒 體 報 道 ， 特 朗 普 在 2009 年 曾 經 對 中 國 出 售 軍 械 ， 導 致 有 關 中 國 國 家 發 生 系 統 性 軍 事 政 變 。 但 是 在 去 年 的 共 和 黨 大 會 中 ， 特 朗 普 與 蔡 英 文 曾 經 在 一 場 談 判 中 指 責 中 國 進 行 侵 略 戰 爭 ， 並 且 引 發 了 民 主 黨 總 統 候 選 人 希 拉 里 · 克 林 頓 在 競 選 過 程 中 提 到 中 國 問 題 。 這 一 說 法 還 伴 隨 著 不 斷 的 不 確 定 性 ， 中 美 兩 個 國 家 如 果 會 以 不 同 的 方 式 進 行 接 觸 ， 就 會 發 生 「 對 立 、 對 抗 及 戰 爭 」 或 者 「 戰 爭 」 。 相 互 指 責 中 國 軍 隊 採 購 了 很 多 武 器 ， 而 且 中 美 兩 國 在 中 國 軍 事 行 動 方 面 「 一 直 有 分 歧 」 ， 「 雙 方 領 導 人 還 會 繼 續 做 生 意 ， 一 方 面 中 美 軍 隊 在 大 家 都 很 關 注 的 時 候 會 做 生 意 ， 另 一 方 面 雙 方 在 雙 方 實 際 上 有 很 多 爭 執 和 問 題 」 。 「 這 樣 的 時 間 比 戰 爭 時 間 短 ， 」 他 說 ， 「 有 人 認 為 中 國 進 行 侵 略 戰 爭 ， 也 有 人 認 為 中 國 軍 方 在 開 戰 前 做 了 很 多 的 行 動 。 」 其 中 一 項 研 究 的 主 要 參 與 者 是 在 美 國 國 安 部 公 約 處 長 彼 得 · 哈 里 斯 （ ） 上 發 表 的 一 份 文 章 。 哈 里 斯 說 ， 「 我 對 中 國 和

In [ ]:
reporter('總統宣布國防預算大漲',max_length=800)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '總統宣布國防預算大漲 。 自 2002 年 中 國 首 次 提 出 抗 日 戰 爭 將 領 軍 國 訓 的 決 定 以 來 ， 中 共 和 國 家 機 密 就 此 受 到 重 視 。 ; 中 國 共 產 黨 發 起 的 「 五 周 抗 戰 」 戰 爭 在 國 共 兩 黨 的 戰 場 上 也 有 發 生 。 中 共 黨 報 刊 登 的 第 一 篇 文 章 也 是 在 「 抗 戰 勝 利 日 」 周 五 中 共 黨 報 刊 登 的 一 篇 《 共 青 團 中 央 國 務 院 抗 戰 事 業 發 展 研 究 所 」 的 文 章 中 提 到 ： 「 歷 代 領 導 和 領 導 人 都 認 為 ， 抗 日 戰 爭 是 中 華 民 族 偉 大 復 興 的 最 高 層 級 戰 略 目 標 和 最 艱 巨 的 任 務 。 從 歷 史 角 度 看 ， 中 國 人 民 抗 日 戰 爭 的 勝 利 ， 充 分 證 明 了 中 華 民 族 歷 史 上 的 鬥 爭 精 神 ， 是 中 華 民 族 歷 史 上 最 堅 定 不 移 的 反 法 西 斯 主 義 和 和 平 偉 大 復 興 的 堅 定 決 心 。 」 中 共 歷 史 上 就 有 兩 次 「 黨 國 宣 傳 」 ， 都 反 對 民 族 分 裂 和 中 國 獨 立 。 而 就 在 中 國 第 一 次 「 革 命 」 之 際 ， 中 國 卻 發 生 了 一 場 「 大 換 血 」 。 在 「 九 月 事 變 」 週 年 之 際 ， 中 國 共 產 黨 在 其 官 方 媒 體 上 公 布 了 一 則 關 於 當 年 「 九 月 事 變 」 的 通 知 。 「 九 月 事 變 」 當 天 ， 中 共 官 方 媒 體 《 人 民 日 報 海 外 版 》 刊 登 了 一 則 通 知 ， 在 當 時 中 共 歷 史 上 「 最 後 一 次 、 最 重 要 的 一 個 連 任 」 時 期 ， 「 九 月 事 變 」 讓 領 導 層 以 軍 情 部 署 的 方 式 決 定 了 當 年 備 在 廣 西 南 寧 的 軍 情 部 署 。 該 通 知 從 通 知 的 一 頁 就 能 看 出 ， 「 九 月 事 變 」 一 字 之 蔽 。 除 了 通 知 的 第 一 頁 ， 第 三 頁 的 文 字 和 「 紅 色 革 命

In [ ]:
for unit in test[:15]:
  input = unit['title']+"BEG;END"
  output = reporter(input,max_length=150)[0]['generated_text'].replace("BEG;END"," -> \t")
  print(f"{output}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


怎樣才能重新熱愛自己的工作？ -> 	 但 你 可 否 通 過 這 些 方 式 來 讓 你 有 更 多 時 間 思 考 工 程 的 全 貌 ？ 我 們 都 知 道 應 該 如 何 做 才 能 更 好 地 熱 愛 它 。 那 麼 ， 如 何 利 用 如 蘋 果 這 樣 的 公 司 來 重 溫 童 年 時 代 ， 以 及 它 的 經 典 產 品 ， 是 否 也 有 其 它 值 得 關 注 的 地 方 呢 ？ 在 今 年 的 10 月 ， 科 技 公 司 創 始 人 馬 克 · 扎 克 伯 格 （ ） 接 受 專 訪 談 到 了 自 己 的 想 法 ： 我


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


台陸委會主委因張顯耀獲不起訴辭職下台 -> 	 張 顯 耀 曾 被 指 參 加 「 六 四 」 前 夕 的 「 天 安 門 六 · 四 民 運 」 的 聲 援 ， 但 未 獲 批 准 下 台 。 對 此 ， 香 港 媒 體 引 述 蔡 英 文 政 府 高 層 的 對 話 時 指 出 ， 張 顯 耀 「 已 經 盡 力 」 ， 已 經 盡 力 在 兩 岸 關 係 方 面 發 揮 「 決 策 和 組 織 」 的 重 要 性 。 蔡 英 文 的 立 場 相 比 於 國 民 黨 立 場 明 確 ， 認 為 張 顯 耀 是 否 參 加 國 民 黨 與 國 民 黨 的


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

解析被毛澤東稱為「第一等聖人」的魯迅 -> 	 魯 迅 在 中 國 文 壇 極 重 要 的 地 位 與 英 國 名 人 雅 各 布 · 盧 共 同 點 在 於 他 在 文 學 、 歷 史 以 及 社 會 哲 學 方 面 與 日 本 的 大 量 學 者 和 學 者 有 密 切 聯 繫 而 且 他 對 日 本 歷 史 上 的 種 族 主 義 主 題 和 文 學 史 、 政 治 主 題 做 出 了 深 入 的 思 考 和 探 索 。 魯 迅 先 生 在 《 文 學 的 奇 妙 歸 宿 》 一 書 中 指 出 ， 他 在 1936 年 發 表 的 《 魯 迅 的 文 學


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


塔利班發佈視頻稱綁架了一名中國遊客 -> 	 中 國 遊 客 被 抓 後 曾 在 一 家 機 場 前 往 聖 尼 古 拉 ， 以 躲 避 暴 恐 。 法 新 社 引 述 參 與 在 阿 富 汗 的 一 名 中 國 遊 客 稱 ， 他 曾 因 涉 嫌 違 反 聯 合 國 制 裁 而 被 禁 止 離 境 和 入 境 。 中 國 外 交 部 網 站 曾 將 這 一 報 道 發 佈 在 《 聯 合 國 海 洋 法 公 約 》 第 40 章 附 件 13 中 ， 引 起 國 際 社 會 廣 泛 關 注 ， 引 發 了 中 國 政 府 高 層 對 這 一 事 件 的 各 種


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


深圳海關截獲大量走私大陸iPhone6 -> 	 嫌 犯 是 涉 嫌 從 香 港 帶 入 內 地 的 「 黃 色 通 道 」 走 私 走 私 的 香 港 旅 客 ， 但 從 未 涉 及 。 截 至 11 月 24 日 ， 已 經 將 走 私 大 陸 貨 物 通 關 案 件 1. 4 萬 4 千 件 ， 同 比 下 降 38. 76 % ， 涉 及 港 貨 總 額 15. 1 億 港 元 。 截 至 目 前 ， 其 最 重 的 1. 4 億 港 元 走 私 品 中 ， 只 有 一 盒 進 口 的 貨 物 。 一 家 工 廠 雇 員 告 訴 記 者 ， 正 如 其 所
你問我答：返鄉路上讓人抓狂的「高科技黃牛」 -> 	 （ 圖 片 來 源 ： ） 根 據 美 國 政 府 官 方 的 報 告 ， 去 年 共 有 逾 3. 4 萬 名 中 國 留 學 生 到 英 國 留 學 ， 這 些 留 學 生 除 了 中 國 學 歷 高 之 外 ， 還 包 括 了 美 國 國 籍 學 生 、 美 國 勞 工 部 的 非 華 裔 僱 主 、 英 國 居 民 、 留 學 生 。 根 據 數 字 ， 截 止 2016 年 12 月 的 9 個 月 ， 英 國 在 校 生 總 數 為 13. 6 萬 人 ， 其 中 包 括 教 師
